In [7]:
!pip -q install langchain sentence-transformers faiss-cpu pandas transformers torch

In [8]:
!ls


archive.zip  sample_data


In [9]:
!unzip archive.zip


Archive:  archive.zip
  inflating: vgsales.csv             


In [10]:
import pandas as pd

df = pd.read_csv("/content/vgsales.csv")
df.head()


,Rank,Name,Platform,Year,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales
0,1,Wii Sports,Wii,2006.0,Sports,Nintendo,41.49,29.02,3.77,8.46,82.74
1,2,Super Mario Bros.,NES,1985.0,Platform,Nintendo,29.08,3.58,6.81,0.77,40.24
2,3,Mario Kart Wii,Wii,2008.0,Racing,Nintendo,15.85,12.88,3.79,3.31,35.82
3,4,Wii Sports Resort,Wii,2009.0,Sports,Nintendo,15.75,11.01,3.28,2.96,33.00
4,5,Pokemon Red/Pokemon Blue,GB,1996.0,Role-Playing,Nintendo,11.27,8.89,10.22,1.00,31.37


In [11]:
import pandas as pd

df = pd.read_csv("/content/vgsales.csv")
df = df.dropna(subset=["Name","Platform","Genre","Publisher","Year","Global_Sales"])
df["Year"] = df["Year"].astype(int)

df.head()


,Rank,Name,Platform,Year,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales
0,1,Wii Sports,Wii,2006,Sports,Nintendo,41.49,29.02,3.77,8.46,82.74
1,2,Super Mario Bros.,NES,1985,Platform,Nintendo,29.08,3.58,6.81,0.77,40.24
2,3,Mario Kart Wii,Wii,2008,Racing,Nintendo,15.85,12.88,3.79,3.31,35.82
3,4,Wii Sports Resort,Wii,2009,Sports,Nintendo,15.75,11.01,3.28,2.96,33.00
4,5,Pokemon Red/Pokemon Blue,GB,1996,Role-Playing,Nintendo,11.27,8.89,10.22,1.00,31.37


In [12]:
!pip -q install langchain langchain-community langchain-core


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 15.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 22.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.0/51.0 kB 2.4 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.32.4, but you have requests 2.32.5 which is incompatible.


In [13]:
from langchain_core.documents import Document


In [14]:
from langchain_core.documents import Document

documents = []
for _, r in df.iterrows():
    text = (
        f"Name: {r['Name']} | Platform: {r['Platform']} | "
        f"Genre: {r['Genre']} | Publisher: {r['Publisher']} | "
        f"Year: {r['Year']} | Global Sales: {r['Global_Sales']} million"
    )
    documents.append(Document(page_content=text))

len(documents)

16291

In [15]:
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import HuggingFaceEmbeddings

In [16]:
from langchain_community.embeddings import HuggingFaceEmbeddings

embeddings = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-MiniLM-L6-v2"
)

/tmp/ipython-input-2671871813.py:3: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the `langchain-huggingface package and should be used instead. To use it run `pip install -U `langchain-huggingface` and import as `from `langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(
/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [18]:
from langchain_community.vectorstores import FAISS

vectorstore = FAISS.from_documents(documents, embeddings)

In [19]:
query = "Which platform has the highest global sales?"
results = vectorstore.similarity_search(query, k=5)

for i, r in enumerate(results, 1):
    print(f"\n--- Source {i} ---")
    print(r.page_content)


--- Source 1 ---
Name: R.U.S.E. | Platform: PC | Genre: Strategy | Publisher: Ubisoft | Year: 2010 | Global Sales: 0.28 million

--- Source 2 ---
Name: R.U.S.E. | Platform: X360 | Genre: Strategy | Publisher: Ubisoft | Year: 2010 | Global Sales: 0.27 million

--- Source 3 ---
Name: EX Monopoly | Platform: GBA | Genre: Misc | Publisher: Takara | Year: 2001 | Global Sales: 0.22 million

--- Source 4 ---
Name: Scaler | Platform: XB | Genre: Platform | Publisher: Global Star | Year: 2004 | Global Sales: 0.04 million

--- Source 5 ---
Name: ParaWorld | Platform: PC | Genre: Strategy | Publisher: Deep Silver | Year: 2006 | Global Sales: 0.02 million


In [20]:
!pip -q install transformers torch

In [26]:
from transformers import pipeline

generator = pipeline("text2text-generation", model="google/flan-t5-base")

config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/990M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

Device set to use cpu


In [27]:
def rag_answer(question, k=5, max_new_tokens=128):
    # 1) Retrieve
    hits = vectorstore.similarity_search(question, k=k)
    context = "\n".join([h.page_content for h in hits])

    # 2) Generate (with constraints)
    prompt = f"""Answer the question using ONLY the information below."

Context:
{context}

Question: {question}
"""

    answer = generator(prompt, max_new_tokens=max_new_tokens)[0]["generated_text"]

    return answer, hits


In [28]:
answer, sources = rag_answer("What are some best-selling sports games after 2010?", k=5)
print(answer)

Winter Sports 3: The Great Tournament | Winter Sports 2011 | Wii | Genre: Sports | Publisher: Tradewest | Year: 2010 | Global Sales: 0.05 million Name: Winter Sports 2011 | Platform: Wii | Genre: Sports | Publisher: Tradewest | Year: 2010 | Global Sales: 0.08 million Name: Winter Sports 2011 | Platform: Wii | Genre: Sports | Publisher: Tradewest | Year: 2010 | Global Sales: 0.08 million Name: Winter Sports 2011 | Platform: Wii | Genre: Sports | Publisher: Tradewest | Year: 2010 | Global Sales: 0.08 million


In [31]:
for i, s in enumerate(sources, 1):
  print(f"\n[Source {i}] {s.page_content}")


[Source 1] Name: Winter Sports 3: The Great Tournament | Platform: Wii | Genre: Sports | Publisher: Tradewest | Year: 2010 | Global Sales: 0.05 million

[Source 2] Name: Winter Sports 2011 | Platform: Wii | Genre: Sports | Publisher: DTP Entertainment | Year: 2011 | Global Sales: 0.08 million

[Source 3] Name: Winter Sports 2011 | Platform: PS3 | Genre: Sports | Publisher: DTP Entertainment | Year: 2011 | Global Sales: 0.05 million

[Source 4] Name: Sports Collection | Platform: DS | Genre: Sports | Publisher: Ubisoft | Year: 2010 | Global Sales: 0.08 million

[Source 5] Name: MLB 10: The Show | Platform: PS2 | Genre: Sports | Publisher: Sony Computer Entertainment | Year: 2010 | Global Sales: 0.41 million
